In [2]:
import os
from pathlib import Path
import subprocess
import json

import pandas as pd
from notion_client import Client
from transformers import pipeline

from config import (
    CANDIDATE_LABELS,
    DATABASE_ID,
    NOTION_ACCESS_TOKEN,
    GYAZO_ACCESS_TOKEN,
    CACHE_PATH,
)
from lib import get_web_content, label_text, post_to_notion

In [ ]:
URL = "https://arxiv.org/abs/2301.02654"

In [ ]:
notion = Client(auth=NOTION_ACCESS_TOKEN)

In [ ]:
p = get_web_content(url=URL, cache_path=CACHE_PATH, arxiv_categories=ARXIV_CATEGORIES)

In [ ]:
import arxiv
from bs4 import BeautifulSoup

In [ ]:
translator = pipeline('translation', model='staka/fugumt-en-ja')

In [ ]:
soup = BeautifulSoup(p.html_content, "html.parser")

figures = []
for fig in soup.find_all("figure"):
    if not len(fig.find_all("td")) > 0:
        figures.append(fig)

In [ ]:
abstract = next(arxiv.Search(id_list=[URL.split("/")[-1]]).results()).summary
translated_abstract = translator(abstract)

In [ ]:
html = f"""
<html>
    <body>
        <h2>Abstract</h2>
            <p>{translated_abstract}</p>
        <h2>Figures</h2>
        {''.join(list(map(lambda x: str(x), figures)))}
    </body>
</html>
"""

In [ ]:
with open("/home/workspace/notion-auto-archive/content/i.html", "w") as f:
    f.write(html)

In [ ]:
if p.tags is None:
    p.tags = label_text(
        text=p.cleansed_content,
        candidate_labels=CANDIDATE_LABELS,
        threshold=0.9,
    )

In [ ]:
p.tags

In [ ]:
# import pickle
# pickle.dump(p, open("p.pkl", "wb"))

# import pickle
# p = pickle.loads(open("p.pkl", "rb").read())

In [ ]:
post_to_notion(
    notion_access_token=NOTION_ACCESS_TOKEN,
    gyazo_access_token=GYAZO_ACCESS_TOKEN,
    database_id=DATABASE_ID,
    processed_content=p,
)